# 라이브러리 설치 및 모듈 삽입 등

In [1]:
from flask import Flask, request
import pymysql as ps
import db

NameError: name 'true' is not defined

# DB와 FLASK 연결

In [5]:
conn = ps.connect(host='localhost', user='root', passwd='12345', db='aqu4men') 
#DB연결 (호스트, 사용자명, 비밀번호, 데이터베이스) MariaDB host내 DB와 직접 연결
#user: user name / passwd: 설정한 패스워드 / host: DB가 존재하는 host / db: 연결할 데이터베이스 이름 / charset: 인코딩 설정

# 커서 생성
- 연결한 DB와 상호작용하기 위해 cursor 객체를 생성 

In [ ]:
curs = conn.cursor()  #일반 cursor는 튜플형태로 값 반환)
#cursor = juso_db.cursor(pymysql.cursors.DictCursor) <- 데이터프레임 형태로 결과를 쉽게 변환할 수 있도록 딕셔너리 형태로 결과를 반환해주는 DictCursor

# 데이터 조작
- PyMySQL의 모든 데이터 조작은 execute()와 fetch() 계열을 사용
- fetchall() : 모든 데이터를 한 번에 가져올 때 사용
- fetchone() : 한 번 호출에 하나의 행만 가져올 때 사용
- fetchmany(n) : n개만큼의 데이터를 가져올 때 사용

## 데이터 조회
     sql = "SELECT * FROM `busan-jibun`;"
     cursor.execute(sql)
     result = cursor.fetchall()
- SELECT 명령을 위해 SQL 문을 따로 변수에 넣어주고 cursor.execute(sql) 을 사용해 SQL를 실행합니다. 실행한 결과를 fetchall()을 이용해 받아옵니다.

## 데이터 삽입
    sql = '''INSERT INTO `busan-jibun` (관리번호, 일련번호, 시도명, 시군구명) 
           VALUES ('1234567891234567891234567', '1', '서울특별시', '강남구');'''
    cursor.execute(sql)
    juso_db.commit()
    
- execute()를 이용해 INSERT 쿼리를 실행한 후, commit()을 날려줍니다.
- 커밋을 날리지 않으면 execute()를 아무리 해도 결과가 DB에 반영되지 않습니다
- 참고로, fetchone() 함수를 사용한 후 pandas를 사용해 결과를 데이터프레임으로 변경하기 위해서는
    result = pd.DataFrame(result, index=[0]) 로 인덱스를 명시해주어야 에러가 나지 않고 올바르게 리턴됩니다.

## 데이터 변경
    sql = '''UPDATE `busan-jibun`
      SET 시도명 = '부산광역시', 시군구명 = '해운대구'
      WHERE 시도명 != '부산광역시';'''
    cursor.execute(sql)
    juso_db.commit()

- 

## 데이터 삭제
    sql = '''DELETE FROM `busan-jibun`
      WHERE `법정동코드` IS NULL;'''
    cursor.execute(sql)
    juso_db.commit()
- 

## execute()/executemany()에 Placeholder 사용하기
1. Placeholder란? 
- 동적 SQL문을 구성할 때 중요하게 사용할 수 있는 기능
- 동적 값이 들어갈 위치에 %s를 이용해 SQL문을 만들어 놓고,
- execute() 계열 메서드 첫번째 파라미터에는 SQL을, 두번째 파라미터에는 실제 데이터를 리스트나 튜플 형태로 넣어주면 된다.

        excute(SQL, a-data)
        executemany(SQL, multiple-data)
- 하나의 데이터에만 적용시키려면 execute()
- 여러개의 데이터를 한 번에 대량으로 적용시키려면 executemany()

2. Placeholder 특징
- 두번째 파라미터에 들어간 데이터 순서대로 SQL이 적용되고,
- 특히 문자의 경우 따옴표 등의 특수문자들이 자동으로 이스케이프 되어 처리됨
- 문자열, 숫자 등에 관계 없이 대치할 값은 모두 %s로 쓰인다. (일반 문자열에서 사용하는 %s, %d와는 다름)
- %s는 칼럼 값을 대치할 때만 사용할 수 있음


In [ ]:
def inser_db(data):
    sql = f"insert into test(sensing) values ('{data}')"
    curs.execute(sql)
    conn.commit()
    
def select_db():
    sql = "select * from test"
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    return result

In [16]:
app = Flask(__name__)

@app.route('/')
def root():
    return 'mainpage'

@app.route('/insert', methods=['GET'])
def insert():
    if request.method == 'GET':
        data = request.args['data']
        
        print(data)
        insert_db(data)
    return '데이터 삽입 성공'

@app.route('/select')
def select():
    result = select_db()
    print(result)
    return '데이터 출력 성공'

if __name__ == '__main__':
    app.run(host='192.168.30.47', port=5025)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.30.47:5025/ (Press CTRL+C to quit)
192.168.30.47 - - [16/Aug/2022 16:55:12] "GET / HTTP/1.1" 200 -
